## Import packages

In [1]:
%matplotlib widget

In [2]:
import os
import glob
import numpy  as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import ipywidgets as widgets

### Choose file to inspect

In [6]:
directory='/Users/saroltagabulya/git/Orca/'
wavs=glob.glob('*.wav')
file=[]

dropdown=widgets.Dropdown(
    options=wavs,
    description='Wav files:',
    disabled=False,
)

def dropdown_eventhandler(change):
    file.clear()
    file.append(change.new)
    print(change.new)
dropdown.observe(dropdown_eventhandler, names='value')
display(dropdown)

Dropdown(description='Wav files:', options=('2015-11-10--15-25.wav', '2017-02-04--10-14-06--00-16-35--BC.wav',…

whale_cropped_test.wav


### Read in wav file

In [8]:
# Read in wav
filename = directory + file[0]
y, sr = librosa.load(filename)
print('Number of data points: {}, fs: {} Hz, resulting file length: {} min.'.format(np.shape(y)[0], sr, np.shape(y)[0]/sr/60))

Number of data points: 1166478, fs: 22050 Hz, resulting file length: 0.8816916099773242 min.


## Display wav file

In [9]:
# Display wav
plt.figure(figsize=(20, 5))
#librosa.display.waveshow(y, sr)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Calculate spectogram

In [10]:
# Convert to spectogram 
Y = librosa.stft(y)
Ydb = librosa.amplitude_to_db(abs(Y))

### Plot spectrogram

In [11]:
#Normalize spectrogram
min_value=pd.DataFrame(Ydb).min().min()
Ydb_norm=np.array(pd.DataFrame(Ydb)-min_value)

In [12]:
#Whiten? (extract average background noise for each buffer)
mean_values=pd.DataFrame(Ydb_norm).mean()
Ydb_norm_w=np.array(pd.DataFrame(Ydb_norm).apply(lambda x: x-mean_values, axis=1))

In [13]:
# Plot spectrogram
plt.figure(figsize=(200, 5))
librosa.display.specshow(Ydb_norm, cmap='seismic', sr=sr, x_axis='time', y_axis='hz')
plt.xticks(np.arange(0, len(Ydb_norm)+1, 10))
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
plt.close()

### Event handling function

In [15]:
def onclick(event):
    if event.key=='b':
        row.clear()
        events.append(event)
        row.append(event.xdata)
        plt.gca().set_title('onset: {}'.format(event.xdata))
        
    elif event.key=='e':
        events.append(event)
        row.append(event.xdata)
        plt.gca().set_title('offset: {}'.format(event.xdata))
        
        if len(row)==2:
            row.append(filename)
            time_stamps.loc[len(time_stamps)] = row
            plt.gca().set_title('Timestamps appended. \n onset: {} \n offset {}'.format(onset, offset))
            row.clear()
            
    elif event.key == 's':
        # save croppings
        print('saved dataframe')
        time_stamps.to_csv('time_stamps.csv')
        
    else:
        plt.gca().set_title('Could not interpret user input, please make a new selection:\n b: define onset timestamps \n e: define offset timestamp and add to df \n s: save df to csv file'.format(onset, offset))

### Initializing data structures

In [16]:
time_stamps=pd.DataFrame(columns=['onset', 'offset', 'filename'])
events = []
row=[]

### Plotting and connecting eventhandler to plot

In [17]:
# Plot spectrogram
plt.figure(figsize=(150, 5))
# tell mpl_connect we want to pass a 'button_press_event' into onclick when the event is detected
plt.gcf().canvas.mpl_connect('key_press_event', onclick)
librosa.display.specshow(Ydb_norm, cmap='seismic', sr=sr, x_axis='time', y_axis='hz')
plt.xticks(np.arange(0, len(Ydb_norm)+1, 10))
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
time_stamps

,onset,offset,filename
0,12.761527,15.393785,/Users/saroltagabulya/git/Orca/whale_cropped_t...
1,39.193785,44.019592,/Users/saroltagabulya/git/Orca/whale_cropped_t...


### Make sure to save time_stamps as csv by clicking s on graph!